# Python library for the analysis of raw data from the SCiO

Note: The SCiO spectrometer is manufactured by Consumer Physics

### Questions regarding decoding:

- How many datapoints are there? 331 or 400?
- What are the remaining data bytes?
- Is the data encrypted?
- Is the data gzipped?


In [ ]:
import json
import struct
import logging
import datetime
import os
import csv
import base64
import serial
import serial.tools.list_ports as pyserial
import numpy as np

# Logging/output format setup
log = logging.getLogger('root')
log.setLevel(logging.DEBUG)
logging.basicConfig(format='[%(asctime)s] %(levelname)8s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [ ]:
class scio_analyse:
    def __init__(self):
        self.device_info = {}
        self.scan_rawdata = {}
        self.calibration_data = {}
        self.key = b''
    
    def parse_scan(self):
        # Convert to bytes
        sample_bytes          = base64.urlsafe_b64decode(self.scan_rawdata['sample']) 
        sample_dark_bytes     = base64.urlsafe_b64decode(self.scan_rawdata['sample_dark'])
        sample_gradient_bytes = base64.urlsafe_b64decode(self.scan_rawdata['sample_gradient'])
        
        # Same for calibration
        sample_cal_bytes          = base64.urlsafe_b64decode(self.calibration_data['sample']) 
        sample_dark_cal_bytes     = base64.urlsafe_b64decode(self.calibration_data['sample_dark'])
        sample_gradient_cal_bytes = base64.urlsafe_b64decode(self.calibration_data['sample_gradient'])
        
        # Decode
        sample_raw          = np.array(struct.unpack('>4x400I196x', sample_bytes))
        sample_dark_raw     = np.array(struct.unpack('>4x400I196x', sample_dark_bytes))
        sample_gradient_raw = np.array(struct.unpack('>4x400I52x',  sample_gradient_bytes))
        sample_cal          = np.array(struct.unpack('>4x400I196x', sample_cal_bytes))
        sample_dark_cal     = np.array(struct.unpack('>4x400I196x', sample_dark_cal_bytes))
        sample_gradient_cal = np.array(struct.unpack('>4x400I52x',  sample_gradient_cal_bytes))
        
        # Calibration
        sample          = sample_raw - sample_cal
        sample_dark     = sample_dark_raw - sample_dark_cal
        sample_gradient = sample_gradient_raw - sample_gradient_cal
        
        # Calculate reflectance
        corrected_sample   = sample - sample_dark
        corrected_gradient = sample_gradient - sample_dark
        reflectance        = corrected_sample / corrected_gradient
        
        return(reflectance)
    
    def load_calibration_file(self, calibration_fn = 'calibration.json'):
        info, data = self.load_file(calibration_fn)
        # Store information
        self.device_info = info
        self.calibration_data = data
        return
    
    def load_data_file(self, data_fn):
        info, data = self.load_file(data_fn)
        # Store information
        self.device_info = info
        self.scan_rawdata = data
        return
    
    def load_file(self, fn):
        # Load the JSON file
        with open(fn, 'r') as file:
            json_data = json.load(file)

        # Load the "scan" object
        data = json_data['scan']
        
        # Load the device information object
        info = json_data['device']
        
        return(info, data)
    
    def run_parsing(self, data_fn, calibration_fn, silent=True):
        # Load calibration file
        self.load_calibration_file(calibration_fn)
        # Load data file
        self.load_data_file(data_fn)
        # Parse data
        reflectance = self.parse_scan()
        return(reflectance)
    
    # Generic, for testing
    def test_parse_response(self, response_data_raw):
        # Parse response
        print('As hex:', response_data_raw[2].hex())
        print(len(response_data_raw[2]))
        
        data_struct = '<4I' # File list
        resp = list(struct.unpack(data_struct, response_data_raw[2]))
        print(resp)
        file_list = {}
        for file_nb in range(0, int(len(resp)/2)):
            if((resp[file_nb*2] >= 87) & (resp[file_nb*2] <= 95)):
                file_list.update({file_nb:{'file_type':resp[file_nb*2], 'file_version':resp[file_nb*2+1]}})
                key_start_byte = file_nb*8
                key_end_byte   = file_nb*8+4
                val_start_byte = file_nb*8+4
                val_end_byte   = file_nb*8+8
                attr = {'key':response_data_raw[2][key_start_byte:key_end_byte].hex(),
                        'value':response_data_raw[2][val_start_byte:val_end_byte].hex()}
                file_list.update({file_nb:attr})
            else:
                attr = {'file_type':resp[file_nb*2], 'file_version':resp[file_nb*2+1]}
                file_list.update({file_nb:attr})
            pass
        
        return(resp)

In [ ]:
calibration_fn = './calibration.json'
data_fn        = './scan.json'

# Now set up device
scio = scio_analyse() # Create an instance for this device address

#scio.load_calibration_file()
#print(scio.calibration_data)

refl = scio.run_parsing(data_fn, calibration_fn, silent=False)
print(refl)
print('Done...')